In [1]:
# from prophet import Prophet
import os
import pandas as pd
import numpy as np
import datetime, itertools
# from prophet.plot import plot_yearly, plot_weekly, plot_plotly, plot_components_plotly
# import plotly.graph_objs as go
from sklearn.metrics import mean_squared_error

In [4]:
# import cv2 as cv

# Transform Data

In [7]:
# DATA_DIR = os.path.join("..", "data")
# fp = os.path.join(DATA_DIR, "hydrograph-excel-sheet-tp-cleaned.xlsx")
fp = "hydrograph-excel-sheet-tp-cleaned.xlsx"
xl = pd.ExcelFile(fp)
gages = xl.sheet_names
hydro_data = {s: xl.parse(s) for s in gages}

In [8]:
def flatten_sheet(sheet_name: str, src_data: dict):
    src_df = src_data[sheet_name]
    
    # Check lengths of columns, some contain only notes so will be
    # much less than 100 and need to be dropped. Most columns
    # should have 365/366 values but a few are missing and need to be filled.
    col_lengths = {c: sum(src_df[c].notna()) for c in src_df.columns}
    keep_cols = [c for c, l in col_lengths.items() if l > 100]
    
    # Check columns are all in the correct order to combine:
    assert "time" in keep_cols[0].lower()
    correct_order = {"time": "ft", "ft": "discharge", "discharge": "time"}
    for i, col in enumerate(keep_cols[:-1]):
        next_col = keep_cols[i+1]
        for key in correct_order.keys():
            if key in col.lower():
                should_be = correct_order[key]
                assert should_be in next_col.lower(), sheet_name
    
    # Iterate through columns and collect data:
    data_subsets = list()
    for start_col in range(0, len(keep_cols), 3):
        df_columns = keep_cols[start_col: start_col+3]
        subset = src_df[df_columns]
        rename = dict(zip(subset.columns, ["time", "ft", "m3"]))
        subset = subset.rename(columns=rename).dropna(how="all")
        data_subsets.append(subset)
        
    # Combine to a single df:
    final =  pd.concat(data_subsets).reset_index(drop=True)
    final["gage"] = sheet_name
    return final

In [9]:
all_sheets = list()
for sname in gages:
    all_sheets.append(flatten_sheet(sname, hydro_data)) 
df = pd.concat(all_sheets).reset_index(drop=True)

In [10]:
min_date, max_date = min(df["time"]), max(df["time"])
all_dates = [min_date.to_pydatetime()]
while all_dates[-1] < max_date:
    all_dates.append(all_dates[-1] + datetime.timedelta(days=1))
    
full_index = list(itertools.product(df["gage"].unique(), all_dates))
df = df.set_index(["gage", "time"])
df = df.reindex(full_index)

In [11]:
gage_list = df.index.get_level_values('gage').unique().tolist()

In [12]:
gage_list

['11402000',
 '11318500',
 '11266500',
 '11208000',
 '11202710',
 '11185500',
 '11189500']

In [13]:
gage_ts = df.loc[gage_list[0]].reset_index(drop = False)
gage_ts.rename(columns = {'time':'ds', 'ft': 'y'}, inplace = True)
min_date = gage_ts['ds'].min()
max_date = gage_ts['ds'].max()

In [14]:
df

ft        m3
gage     time                       
11402000 1984-10-01  54.00  1.529110
         1984-10-02  52.00  1.472476
         1984-10-03  49.00  1.387525
         1984-10-04  49.00  1.387525
         1984-10-05  48.00  1.359209
...                    ...       ...
11189500 2018-09-26   2.86  0.080986
         2018-09-27   2.78  0.078721
         2018-09-28   2.99  0.084667
         2018-09-29   3.12  0.088349
         2018-09-30   3.34  0.094578

[86926 rows x 2 columns]

# Generate Multivariate Time Series Data

In [15]:
gage_df = df.loc[gage_list[0]].reset_index(drop = False)[['time','ft']]
gage_df.rename(columns = {'ft':f'ft_{gage_list[0]}'}, inplace = True)
for gage_num in gage_list[1:]:
    new_gage_df =  df.loc[gage_num].reset_index(drop = False)[['time','ft']]
    new_gage_df.rename(columns = {'ft':f'ft_{gage_num}'}, inplace = True)
    gage_df = gage_df.merge(new_gage_df, on = 'time', how = 'outer')

In [16]:
gage_df.head()

,time,ft_11402000,ft_11318500,ft_11266500,ft_11208000,ft_11202710,ft_11185500,ft_11189500
0,1984-10-01,54.0,10.0,53.0,1.7,NaN,256.0,39.0
1,1984-10-02,52.0,12.0,52.0,1.4,NaN,279.0,42.0
2,1984-10-03,49.0,14.0,51.0,1.4,NaN,284.0,45.0
3,1984-10-04,49.0,13.0,49.0,1.4,NaN,291.0,47.0
4,1984-10-05,48.0,14.0,46.0,1.4,NaN,281.0,50.0


In [49]:
# most 4 years data seems to have no missing values
gage_df.iloc[-365 * 4:].isnull().sum()

time           0
ft_11402000    0
ft_11318500    0
ft_11266500    0
ft_11208000    0
ft_11202710    0
ft_11185500    0
ft_11189500    0
dtype: int64

In [50]:
gage_df = gage_df.iloc[-365 * 4:]

## Baseline Prophet Model

In [28]:
selected_params = {'seasonality_mode':'multiplicative'}
horizon = 30

In [31]:
gage_df.rename(columns = {'time':'ds', 'ft_11402000': 'y'}, inplace = True)
gage_train = gage_df.iloc[: -horizon,:]
gage_test = gage_df.iloc[-horizon:,:]
m = Prophet(**selected_params).fit(gage_train) # **best_params
future = m.make_future_dataframe(periods=horizon)
forecast = m.predict(future)
gage_test['yhat_corrected'] = forecast.iloc[-horizon:]['yhat'].apply(lambda x : max(x,0)).values
gage_test['yhat'] = forecast.iloc[-horizon:]['yhat'].values
forecast['yhat_corrected'] = forecast['yhat'].apply(lambda x : max(x,0))

# gage_test.drop(columns = ['m3'], inplace = True)

# if show_plots:
#     fig1 = m.plot_components(forecast)
#     fig2 = m.plot(forecast)

rmse = np.sqrt(mean_squared_error(gage_test['y'], gage_test['yhat_corrected']))

In [32]:
rmse

22.947919918866564

# Multivariate LSTM Seq2Seq Forecasting

In [61]:
import matplotlib.pyplot as plt
import tensorflow as tf
tf.random.set_seed(1234)

In [62]:
# def split_series(series, n_past, n_future):
#   #
#   # n_past ==> no of past observations
#   #
#   # n_future ==> no of future observations 
#   #
#     X, y = list(), list()
#     for window_start in range(len(series)):
#         past_end = window_start + n_past
#         future_end = past_end + n_future
#         if future_end > len(series):
#             break
#         # slicing the past and future parts of the window
#         past, future = series[window_start:past_end, :], series[past_end:future_end, :]
#         X.append(past)
#         y.append(future)
#     return np.array(X), np.array(y)


def transform_seq2seq_data(df, target_col, feature_col, n_past, n_future):

    X_list = []
    y_list = []
    for i in range(n_past,len(df) - n_future):
        y = df.iloc[i:i + n_future][target_col].values.tolist()
        y_list.append(y)

        X = df.iloc[i - n_past: i][feature_cols].values.tolist()
        X_list.append(X)
    return np.array(X_list), np.array(y_list)
    

In [63]:
# feature_cols = ['m3','swe_avg','swe_max','tp_avg','t2m_avg','tp_max','t2m_max','pixel_sum','pixel_mean','pixel_min','pixel_max']
# target_col = 'm3'

# gage_df_test = gage_df.copy()
# X_list = []
# y_list = []
# for i in range(28,len(gage_df_test) - 14):
#     y = gage_df_test.iloc[i:i + 14][target_col].values.tolist()
#     y_list.append(y)
    
#     X = gage_df_test.iloc[i - 28: i][feature_cols].values.tolist()
#     X_list.append(X)
    
    


In [64]:
# np.array(X_list).shape

In [65]:
gage_df = pd.read_csv('11402000_complete_ts.csv')
gage_df.set_index('time', inplace = True)
# gage_df['target'] = gage_df['m3'].shift()
gage_df.head(28)

,m3,swe_avg,swe_max,tp_avg,t2m_avg,tp_max,t2m_max,pixel_sum,pixel_mean,pixel_min,pixel_max
time,,,,,,,,,,,
2010-01-01,1.599902,0.010051,10.0,36.596780,0.277207,54.227730,1.021534,12514.0,49.074510,34.0,61.0
2010-01-02,1.817942,0.003433,6.0,19.432330,2.140078,40.226460,2.642189,12514.0,49.074510,34.0,61.0
2010-01-03,1.758476,0.000796,4.0,0.766869,-2.116478,1.752489,-0.916607,12514.0,49.074510,34.0,61.0
2010-01-04,1.693347,0.000467,4.0,0.016986,-1.963441,0.045573,-0.204727,12514.0,49.074510,34.0,61.0
2010-01-05,1.662199,0.000412,4.0,0.218129,0.024585,0.302439,0.961063,12514.0,49.074510,34.0,61.0
2010-01-06,1.614060,0.000275,3.0,0.852008,-0.169998,1.713131,0.659763,12514.0,49.074510,34.0,61.0
2010-01-07,1.599902,0.000192,3.0,26.847263,1.220284,30.620916,2.069096,12514.0,49.074510,34.0,61.0
2010-01-08,1.608397,0.000110,2.0,15.534594,0.431471,17.719612,1.665302,12514.0,49.074510,34.0,61.0
2010-01-09,1.699011,0.000082,2.0,6.499332,1.175823,7.467758,1.933275,145640.0,44.240580,27.0,82.0


In [66]:
n_past = 28
n_future = 14 


In [67]:
target_gage = '11402000'

In [68]:
train_size = int(len(gage_df) * 0.8)
train_df,test_df = gage_df[:train_size], gage_df[train_size:] 


feature_cols = ['m3','swe_avg','swe_max','tp_avg','t2m_avg','tp_max','t2m_max','pixel_sum','pixel_mean','pixel_min','pixel_max']

train = train_df[feature_cols]
test = test_df[feature_cols]



In [69]:
train.index

Index(['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04', '2010-01-05',
       '2010-01-06', '2010-01-07', '2010-01-08', '2010-01-09', '2010-01-10',
       ...
       '2015-07-30', '2015-07-31', '2015-08-01', '2015-08-02', '2015-08-03',
       '2015-08-04', '2015-08-05', '2015-08-06', '2015-08-07', '2015-08-08'],
      dtype='object', name='time', length=2046)

In [70]:
# Rescaling
from sklearn.preprocessing import MinMaxScaler
train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

<ipython-input-70-9baa6a697004>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-70-9baa6a697004>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-70-9baa6a697004>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [71]:
feature_cols = ['m3','swe_avg','swe_max','tp_avg','t2m_avg','tp_max','t2m_max','pixel_sum','pixel_mean','pixel_min','pixel_max']
X_train, y_train = transform_seq2seq_data(train, target_col = 'm3',
                                          feature_col = feature_cols, 
                                          n_past = 28, 
                                          n_future = 14)

X_test, y_test = transform_seq2seq_data(test, target_col = 'm3',
                                          feature_col = feature_cols, 
                                          n_past = 28, 
                                          n_future = 14)

In [72]:
# num_target_col = 1


# X_train, y_train = split_series(train.values,n_past, n_future)
# y_train = y_train[:,:,0]
# X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
# y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], num_target_col))
# X_test, y_test = split_series(test.values,n_past, n_future)
# y_test = y_test[:,:,0]
# X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
# y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], num_target_col))

In [73]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
num_target_col = 1
n_features = len(feature_cols)

encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_target_col))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 28, 11)]     0           []                               
                                                                                                  
 lstm_8 (LSTM)                  [(None, 100),        44800       ['input_3[0][0]']                
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                                  
 repeat_vector_2 (RepeatVector)  (None, 14, 100)     0           ['lstm_8[0][0]']                 
                                                                                            

In [74]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 0.01 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=2
   
                            # callbacks=[reduce_lr]
                           )

Epoch 1/25
63/63 - 4s - loss: 0.0196 - val_loss: 0.0137 - 4s/epoch - 65ms/step
Epoch 2/25
63/63 - 1s - loss: 0.0048 - val_loss: 0.0136 - 1s/epoch - 18ms/step
Epoch 3/25
63/63 - 1s - loss: 0.0045 - val_loss: 0.0138 - 1s/epoch - 18ms/step
Epoch 4/25
63/63 - 1s - loss: 0.0044 - val_loss: 0.0138 - 1s/epoch - 19ms/step
Epoch 5/25
63/63 - 1s - loss: 0.0044 - val_loss: 0.0131 - 1s/epoch - 18ms/step
Epoch 6/25
63/63 - 1s - loss: 0.0044 - val_loss: 0.0132 - 1s/epoch - 18ms/step
Epoch 7/25
63/63 - 1s - loss: 0.0044 - val_loss: 0.0133 - 1s/epoch - 18ms/step
Epoch 8/25
63/63 - 1s - loss: 0.0043 - val_loss: 0.0130 - 1s/epoch - 18ms/step
Epoch 9/25
63/63 - 1s - loss: 0.0042 - val_loss: 0.0130 - 1s/epoch - 18ms/step
Epoch 10/25
63/63 - 1s - loss: 0.0044 - val_loss: 0.0134 - 1s/epoch - 18ms/step
Epoch 11/25
63/63 - 1s - loss: 0.0043 - val_loss: 0.0148 - 1s/epoch - 18ms/step
Epoch 12/25
63/63 - 1s - loss: 0.0042 - val_loss: 0.0134 - 1s/epoch - 18ms/step
Epoch 13/25
63/63 - 1s - loss: 0.0041 - val_loss:

In [75]:
# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_target_col))(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 28, 11)]     0           []                               
                                                                                                  
 lstm_10 (LSTM)                 [(None, 28, 100),    44800       ['input_4[0][0]']                
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                                  
 lstm_11 (LSTM)                 [(None, 100),        80400       ['lstm_10[0][0]']                
                                 (None, 100),                                               

In [76]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 0.001 * 0.90 ** x)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.Huber())
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,
                            verbose=2,
                            callbacks=[reduce_lr,early_stopping_callback]
                           
                           )

Epoch 1/25
63/63 - 8s - loss: 0.0253 - val_loss: 0.0141 - lr: 0.0010 - 8s/epoch - 133ms/step
Epoch 2/25
63/63 - 2s - loss: 0.0049 - val_loss: 0.0142 - lr: 9.0000e-04 - 2s/epoch - 37ms/step
Epoch 3/25
63/63 - 2s - loss: 0.0046 - val_loss: 0.0143 - lr: 8.1000e-04 - 2s/epoch - 37ms/step
Epoch 4/25
63/63 - 2s - loss: 0.0045 - val_loss: 0.0144 - lr: 7.2900e-04 - 2s/epoch - 36ms/step
Epoch 5/25
63/63 - 2s - loss: 0.0044 - val_loss: 0.0138 - lr: 6.5610e-04 - 2s/epoch - 37ms/step
Epoch 6/25
63/63 - 2s - loss: 0.0044 - val_loss: 0.0137 - lr: 5.9049e-04 - 2s/epoch - 37ms/step
Epoch 7/25
63/63 - 2s - loss: 0.0043 - val_loss: 0.0137 - lr: 5.3144e-04 - 2s/epoch - 37ms/step
Epoch 8/25
63/63 - 2s - loss: 0.0043 - val_loss: 0.0139 - lr: 4.7830e-04 - 2s/epoch - 36ms/step
Epoch 9/25
63/63 - 2s - loss: 0.0042 - val_loss: 0.0137 - lr: 4.3047e-04 - 2s/epoch - 37ms/step
Epoch 10/25
63/63 - 2s - loss: 0.0044 - val_loss: 0.0138 - lr: 3.8742e-04 - 2s/epoch - 36ms/step
Epoch 11/25
63/63 - 2s - loss: 0.0043 - va

In [77]:
pred_e1d1=model_e1d1.predict(X_test)
pred_e2d2=model_e2d2.predict(X_test)

15/15 [==============================] - 1s 13ms/step


## Evaluate Results

In [78]:
pred_e1d1 = pred_e1d1.reshape(y_test.shape[0], y_test.shape[1])
pred_e2d2 = pred_e2d2.reshape(y_test.shape[0], y_test.shape[1])

pred_e1d1_inverse = scalers['scaler_m3'].inverse_transform(pred_e1d1)
pred_e2d2_inverse = scalers['scaler_m3'].inverse_transform(pred_e2d2)
y_test_inverse = scalers['scaler_m3'].inverse_transform(y_test)

In [79]:
y_test_inverse

array([[ 0.30865363,  0.30015857,  0.30015857, ...,  0.32847542,
         0.35962395,  0.36528732],
       [ 0.30015857,  0.30015857,  0.24805558, ...,  0.35962395,
         0.36528732,  0.34546553],
       [ 0.30015857,  0.24805558,  0.26447935, ...,  0.36528732,
         0.34546553,  0.30015857],
       ...,
       [71.07528495, 24.26753753, 14.89466131, ...,  4.67227969,
         4.38911122,  4.10594276],
       [24.26753753, 14.89466131, 11.10020386, ...,  4.38911122,
         4.10594276,  3.93604168],
       [14.89466131, 11.10020386,  9.08970776, ...,  4.10594276,
         3.93604168,  3.79445744]])

In [80]:
pred_e1d1_inverse

array([[-1.8103908e+00, -1.2263874e+00, -9.3075371e-01, ...,
         1.8495232e-01,  2.1352266e-01,  2.4294977e-01],
       [-1.0934491e+00, -1.0562443e+00, -9.7216314e-01, ...,
         1.7142780e-01,  2.1871765e-01,  2.6346225e-01],
       [-7.5225180e-01, -1.0711981e+00, -1.1180240e+00, ...,
        -2.0264307e-02,  3.5388507e-02,  8.7104328e-02],
       ...,
       [ 1.9868876e+01,  1.7269114e+01,  1.4758994e+01, ...,
         7.4834871e+00,  7.5325823e+00,  7.6333728e+00],
       [ 2.8800196e+01,  2.4107794e+01,  1.9865623e+01, ...,
         7.5372882e+00,  7.5969033e+00,  7.7336950e+00],
       [ 2.1664862e+01,  1.8283188e+01,  1.5024749e+01, ...,
         6.0462227e+00,  6.1434269e+00,  6.2971783e+00]], dtype=float32)

In [81]:
def relative_root_mean_squared_error(true, pred):
    num = np.sum(np.square(true - pred))
    den = np.sum(np.square(pred))
    squared_error = num/den
    rrmse_loss = np.sqrt(squared_error)
    return rrmse_loss

In [82]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

In [83]:
pred_e1d1_inverse.reshape(-1)[:28]

array([-1.8103908 , -1.2263874 , -0.9307537 , -0.49775174, -0.19085045,
       -0.02035625,  0.0604398 ,  0.09711816,  0.11794826,  0.13685584,
        0.15898156,  0.18495232,  0.21352266,  0.24294977, -1.0934491 ,
       -1.0562443 , -0.97216314, -0.6396255 , -0.36937746, -0.19723657,
       -0.0943984 , -0.02726061,  0.0255335 ,  0.07439061,  0.12294681,
        0.1714278 ,  0.21871765,  0.26346225], dtype=float32)

In [84]:
# 1st run
final_rmse = np.sqrt(mean_squared_error(y_test_inverse.reshape(-1), pred_e1d1_inverse.reshape(-1)))
final_mape = mean_absolute_percentage_error(y_test_inverse.reshape(-1), pred_e1d1_inverse.reshape(-1))
final_rrmse = relative_root_mean_squared_error(y_test_inverse.reshape(-1), pred_e1d1_inverse.reshape(-1))
                     
print(f'RMSE = {final_rmse}')
print(f'MAPE = {final_mape}')
print(f'RRMSE = {final_rrmse}')

RMSE = 12.119650953513935
MAPE = 1.377658662358193
RRMSE = 1.6816127975189923


In [85]:
# 1nd run
rmse_list = []
mape_list = []
rrmse_list = []

for pred, truth in zip(pred_e1d1_inverse, y_test_inverse):
    rmse_list.append(np.sqrt(mean_squared_error(truth, pred)))
    mape_list.append(mean_absolute_percentage_error(truth, pred))
    rrmse_list.append(relative_root_mean_squared_error(truth, pred))
    
final_rmse = np.mean(rmse_list)
final_mape = np.mean(mape_list)
final_rrmse = np.mean(rrmse_list)

print(f'RMSE = {final_rmse}')
print(f'MAPE = {final_mape}')
print(f'RRMSE = {final_rrmse}')


RMSE = 5.859020443844475
MAPE = 1.377658662358193
RRMSE = 1.380964439144476


In [30]:
# for pred, truth in zip(pred_e1d1_inverse, y_test_inverse):
#     rmse_list.append(np.sqrt(mean_squared_error(truth, pred)))
#     mape_list.append(mean_absolute_percentage_error(truth, pred))
#     rrmse_list.append(relative_root_mean_squared_error(truth, pred))

# 2nd run
final_rmse = np.sqrt(mean_squared_error(y_test_inverse.reshape(-1), pred_e1d1_inverse.reshape(-1)))
final_mape = mean_absolute_percentage_error(y_test_inverse.reshape(-1), pred_e1d1_inverse.reshape(-1))
final_rrmse = relative_root_mean_squared_error(y_test_inverse.reshape(-1), pred_e1d1_inverse.reshape(-1))
                     
print(f'RMSE = {final_rmse}')
print(f'MAPE = {final_mape}')
print(f'RRMSE = {final_rrmse}')

RMSE = 12.182589283034813
MAPE = 1.5755370753380615
RRMSE = 2.2385338903336236


In [31]:
# 2nd run

rmse_list = []
mape_list = []
rrmse_list = []

for pred, truth in zip(pred_e1d1_inverse, y_test_inverse):
    rmse_list.append(np.sqrt(mean_squared_error(truth, pred)))
    mape_list.append(mean_absolute_percentage_error(truth, pred))
    rrmse_list.append(relative_root_mean_squared_error(truth, pred))
    
final_rmse = np.mean(rmse_list)
final_mape = np.mean(mape_list)
final_rrmse = np.mean(rrmse_list)

print(f'RMSE = {final_rmse}')
print(f'MAPE = {final_mape}')
print(f'RRMSE = {final_rrmse}')

    

RMSE = 5.989928050479377
MAPE = 1.5755370753380615
RRMSE = 1.4608263049468748


In [32]:
# reported
rmse_list = []
mape_list = []
rrmse_list = []

for pred, truth in zip(pred_e2d2_inverse, y_test_inverse):
    rmse_list.append(np.sqrt(mean_squared_error(truth, pred)))
    mape_list.append(mean_absolute_percentage_error(truth, pred))
    rrmse_list.append(relative_root_mean_squared_error(truth, pred))
    
final_rmse = np.mean(rmse_list)
final_mape = np.mean(mape_list)
final_rrmse = np.mean(rrmse_list)

print(f'RMSE = {final_rmse}')
print(f'MAPE = {final_mape}')
print(f'RRMSE = {final_rrmse}')

RMSE = 5.770056141377242
MAPE = 1.1326347326031592
RRMSE = 1.5695356506080462


In [59]:
# for index,i in enumerate(train.columns):
#     scaler = scalers['scaler_'+i]
#     # pred1_e1d1[:,:,index]=scaler.inverse_transform(pred1_e1d1[:,:,index])
#     pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
#     # pred1_e2d2[:,:,index]=scaler.inverse_transform(pred1_e2d2[:,:,index])
#     pred_e2d2[:,:,index]=scaler.inverse_transform(pred_e2d2[:,:,index])
#     y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
#     y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [33]:
# pred_e1d1.shape

In [34]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# for index,i in enumerate([target_gage]):
#     print(i)
#     for j in range(1,6):
#         print("Day ",j,":")
#         print("MAE-E1D1 : ",mean_absolute_error(y_test[:,j-1,index],pred_e1d1[:,j-1,index]),end=", ")
#         print("MAE-E2D2 : ",mean_absolute_error(y_test[:,j-1,index],pred_e2d2[:,j-1,index]))
#         print("RMSE-E1D1 : ",np.sqrt(mean_squared_error(y_test[:,j-1,index],pred_e1d1[:,j-1,index])),end=", ")
#         print("RMSE-E2D2 : ",np.sqrt(mean_squared_error(y_test[:,j-1,index],pred_e2d2[:,j-1,index])))
        
#     print()
#     print()

In [189]:
# biweekly_error = []
# for pred, ground_truth in zip(pred_e1d1.reshape(pred_e1d1.shape[0],pred_e1d1.shape[1]),
#                               y_test.reshape(y_test.shape[0],y_test.shape[1])):
#     rmse = np.sqrt(mean_squared_error(pred, ground_truth))
#     biweekly_error.append(rmse)

In [194]:
# import seaborn as sns

In [36]:
# sns.histplot(biweekly_error)

In [37]:
# sampled_biweekly_error = [n for n in biweekly_error if n < 100]
# sns.histplot(sampled_biweekly_error)

# Multivariate LSTM Single-Step Forecasting

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
# tf.random.set_seed(7)

In [51]:
feature_cols = ['m3','swe_avg','swe_max','tp_avg','t2m_avg','tp_max','t2m_max','pixel_sum','pixel_mean','pixel_min','pixel_max']
X_train, y_train = transform_seq2seq_data(train, target_col = 'm3',
                                          feature_col = feature_cols, 
                                          n_past = 28, 
                                          n_future = 1)

X_test, y_test = transform_seq2seq_data(test, target_col = 'm3',
                                          feature_col = feature_cols, 
                                          n_past = 28, 
                                          n_future = 1)

y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

In [52]:
n_past = 28

In [53]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(n_past,11)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=20, batch_size=4, verbose=2)

Epoch 1/20
505/505 - 4s - loss: 0.0451 - 4s/epoch - 8ms/step
Epoch 2/20
505/505 - 3s - loss: 0.0105 - 3s/epoch - 5ms/step
Epoch 3/20
505/505 - 3s - loss: 0.0092 - 3s/epoch - 5ms/step
Epoch 4/20
505/505 - 3s - loss: 0.0082 - 3s/epoch - 5ms/step
Epoch 5/20
505/505 - 3s - loss: 0.0075 - 3s/epoch - 5ms/step
Epoch 6/20
505/505 - 3s - loss: 0.0070 - 3s/epoch - 5ms/step
Epoch 7/20
505/505 - 3s - loss: 0.0069 - 3s/epoch - 6ms/step
Epoch 8/20
505/505 - 3s - loss: 0.0065 - 3s/epoch - 6ms/step
Epoch 9/20
505/505 - 3s - loss: 0.0064 - 3s/epoch - 5ms/step
Epoch 10/20
505/505 - 3s - loss: 0.0063 - 3s/epoch - 5ms/step
Epoch 11/20
505/505 - 3s - loss: 0.0060 - 3s/epoch - 5ms/step
Epoch 12/20
505/505 - 3s - loss: 0.0059 - 3s/epoch - 6ms/step
Epoch 13/20
505/505 - 3s - loss: 0.0058 - 3s/epoch - 5ms/step
Epoch 14/20
505/505 - 3s - loss: 0.0058 - 3s/epoch - 5ms/step
Epoch 15/20
505/505 - 3s - loss: 0.0057 - 3s/epoch - 6ms/step
Epoch 16/20
505/505 - 3s - loss: 0.0056 - 3s/epoch - 5ms/step
Epoch 17/20
505/5

In [54]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 4)                 256       
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________


In [55]:
single_step_lstm_pred = model.predict(X_test)

16/16 [==============================] - 0s 2ms/step


In [56]:
single_step_lstm_pred = single_step_lstm_pred.reshape(y_test.shape[0])


single_step_lstm_pred_inverse = scalers['scaler_m3'].inverse_transform([single_step_lstm_pred])
y_test_inverse = scalers['scaler_m3'].inverse_transform([y_test])

In [57]:
final_rmse = np.sqrt(mean_squared_error(y_test_inverse, single_step_lstm_pred_inverse))
final_mape = mean_absolute_percentage_error(y_test_inverse, single_step_lstm_pred_inverse)
final_rrmse = relative_root_mean_squared_error(y_test_inverse, single_step_lstm_pred_inverse)

print(f'RMSE = {final_rmse}')
print(f'MAPE = {final_mape}')
print(f'RRMSE = {final_rrmse}')

RMSE = 0.11770290937528494
MAPE = 0.2031870309762798
RRMSE = 0.12671778208413098


In [ ]:
# def transform_single_step_data(df, target_col, feature_col, n_past, n_future = 1):

#     X_list = []
#     y_list = []
#     for i in range(n_past,len(df) - n_future):
#         y = df.iloc[i:i + 14][target_col].values.tolist()
#         y_list.append(y)

#         X = df.iloc[i - n_past: i][feature_cols].values.tolist()
#         X_list.append(X)
#     return np.array(X_list), np.array(y_list)

In [86]:
feature_cols = ['m3','swe_avg','swe_max','tp_avg','t2m_avg','tp_max','t2m_max','pixel_sum','pixel_mean','pixel_min','pixel_max']
X_train, y_train = transform_seq2seq_data(train, target_col = 'm3',
                                          feature_col = feature_cols, 
                                          n_past = 28, 
                                          n_future = 1)

X_test, y_test = transform_seq2seq_data(test, target_col = 'm3',
                                          feature_col = feature_cols, 
                                          n_past = 28, 
                                          n_future = 1)

y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)


In [132]:
train.iloc[:28]

,m3,swe_avg,swe_max,tp_avg,t2m_avg,tp_max,t2m_max,pixel_sum,pixel_mean,pixel_min,pixel_max
time,,,,,,,,,,,
2010-01-01,-0.980240,-0.999977,-0.992416,-0.932862,-0.269596,-0.917210,-0.284100,-0.981680,-0.710263,-0.591837,-0.925203
2010-01-02,-0.977130,-0.999992,-0.995449,-0.964351,-0.184114,-0.938586,-0.211043,-0.981680,-0.710263,-0.591837,-0.925203
2010-01-03,-0.977978,-0.999998,-0.996966,-0.998593,-0.379435,-0.997324,-0.371469,-0.981680,-0.710263,-0.591837,-0.925203
2010-01-04,-0.978907,-0.999999,-0.996966,-0.999969,-0.372413,-0.999930,-0.339378,-0.981680,-0.710263,-0.591837,-0.925203
2010-01-05,-0.979352,-0.999999,-0.996966,-0.999600,-0.281188,-0.999538,-0.286826,-0.981680,-0.710263,-0.591837,-0.925203
2010-01-06,-0.980038,-0.999999,-0.997725,-0.998437,-0.290117,-0.997385,-0.300408,-0.981680,-0.710263,-0.591837,-0.925203
2010-01-07,-0.980240,-1.000000,-0.997725,-0.950748,-0.226321,-0.953251,-0.236877,-0.981680,-0.710263,-0.591837,-0.925203
2010-01-08,-0.980119,-1.000000,-0.998483,-0.971501,-0.262517,-0.972947,-0.255080,-0.981680,-0.710263,-0.591837,-0.925203
2010-01-09,-0.978827,-1.000000,-0.998483,-0.988077,-0.228361,-0.988599,-0.243000,-0.786785,-0.755045,-0.687075,-0.856911


In [131]:
X_train[0]

array([[-0.98024   , -0.99997694, -0.99241562, -0.93286242, -0.26959613,
        -0.91720958, -0.28409993, -0.98167968, -0.71026266, -0.59183673,
        -0.92520325],
       [-0.97713042, -0.99999213, -0.99544937, -0.96435097, -0.18411436,
        -0.93858556, -0.21104293, -0.98167968, -0.71026266, -0.59183673,
        -0.92520325],
       [-0.97797849, -0.99999817, -0.99696625, -0.99859316, -0.37943538,
        -0.99732444, -0.37146876, -0.98167968, -0.71026266, -0.59183673,
        -0.92520325],
       [-0.97890732, -0.99999893, -0.99696625, -0.99996884, -0.37241293,
        -0.99993042, -0.33937816, -0.98167968, -0.71026266, -0.59183673,
        -0.92520325],
       [-0.97935155, -0.99999906, -0.99696625, -0.99959984, -0.28118819,
        -0.99953826, -0.28682589, -0.98167968, -0.71026266, -0.59183673,
        -0.92520325],
       [-0.98003808, -0.99999937, -0.99772469, -0.99843697, -0.29011704,
        -0.99738453, -0.30040808, -0.98167968, -0.71026266, -0.59183673,
        -0.925

In [100]:
num_target_col = 1
n_past = 28
n_future = 1
n_features = len(feature_cols)

encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_target_col))(decoder_l1)

#
single_step_lstm = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
single_step_lstm.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 28, 11)]     0           []                               
                                                                                                  
 lstm_14 (LSTM)                 [(None, 100),        44800       ['input_7[0][0]']                
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                                  
 repeat_vector_6 (RepeatVector)  (None, 1, 100)      0           ['lstm_14[0][0]']                
                                                                                            

In [101]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 0.01 * 0.90 ** x)
single_step_lstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.Huber())
history_single_step_lstm=single_step_lstm.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=2
                            # callbacks=[reduce_lr]
                           )

Epoch 1/25
64/64 - 5s - loss: 0.0290 - val_loss: 0.0158 - 5s/epoch - 72ms/step
Epoch 2/25
64/64 - 1s - loss: 0.0088 - val_loss: 0.0155 - 991ms/epoch - 15ms/step
Epoch 3/25
64/64 - 1s - loss: 0.0093 - val_loss: 0.0156 - 988ms/epoch - 15ms/step
Epoch 4/25
64/64 - 1s - loss: 0.0084 - val_loss: 0.0156 - 989ms/epoch - 15ms/step
Epoch 5/25
64/64 - 1s - loss: 0.0085 - val_loss: 0.0163 - 977ms/epoch - 15ms/step
Epoch 6/25
64/64 - 1s - loss: 0.0084 - val_loss: 0.0160 - 967ms/epoch - 15ms/step
Epoch 7/25
64/64 - 1s - loss: 0.0083 - val_loss: 0.0164 - 985ms/epoch - 15ms/step
Epoch 8/25
64/64 - 1s - loss: 0.0082 - val_loss: 0.0168 - 981ms/epoch - 15ms/step
Epoch 9/25
64/64 - 1s - loss: 0.0082 - val_loss: 0.0177 - 975ms/epoch - 15ms/step
Epoch 10/25
64/64 - 1s - loss: 0.0087 - val_loss: 0.0158 - 985ms/epoch - 15ms/step
Epoch 11/25
64/64 - 1s - loss: 0.0081 - val_loss: 0.0163 - 990ms/epoch - 15ms/step
Epoch 12/25
64/64 - 1s - loss: 0.0088 - val_loss: 0.0171 - 979ms/epoch - 15ms/step
Epoch 13/25
64/6

In [110]:
pred_single_step = single_step_lstm.predict(X_test)
pred_single_step = pred_single_step.reshape(y_test.shape[0])

16/16 [==============================] - 1s 5ms/step


In [117]:
pred_single_step_inverse = scalers['scaler_m3'].inverse_transform([pred_single_step])
y_test_inverse = scalers['scaler_m3'].inverse_transform([y_test])

In [119]:
rmse_list = []
mape_list = []
rrmse_list = []

for pred, truth in zip(pred_single_step_inverse, y_test_inverse):
    rmse_list.append(np.sqrt(mean_squared_error(truth, pred)))
    mape_list.append(mean_absolute_percentage_error(truth, pred))
    rrmse_list.append(relative_root_mean_squared_error(truth, pred))
    
final_rmse = np.mean(rmse_list)
final_mape = np.mean(mape_list)
final_rrmse = np.mean(rrmse_list)

print(f'RMSE = {final_rmse}')
print(f'MAPE = {final_mape}')
print(f'RRMSE = {final_rrmse}')

RMSE = 12.6423502459464
MAPE = 5.925804591904566
RRMSE = 1.673161707339849


In [123]:
np.sqrt(mean_squared_error(y_test_inverse, pred_single_step_inverse))

12.6423502459464

In [ ]:
RMSE = 5.884456679626631
MAPE = 1.9748009506470705
RRMSE = 0.9109061023635227

In [66]:
gage_df = df.loc[gage_list[0]].reset_index(drop = False)[['time','ft']]
gage_df.rename(columns = {'ft':f'ft_{gage_list[0]}'}, inplace = True)
for gage_num in gage_list[1:]:
    new_gage_df =  df.loc[gage_num].reset_index(drop = False)[['time','ft']]
    new_gage_df.rename(columns = {'ft':f'ft_{gage_num}'}, inplace = True)
    gage_df = gage_df.merge(new_gage_df, on = 'time', how = 'outer')
    
gage_df = gage_df.iloc[-365 * 4:]

In [69]:
target_col = 'ft_11402000'

In [70]:
gage_df['target'] = gage_df[target_col]

In [71]:
time_shift = 30
target_data = gage_df['target'].shift(time_shift)
data = gage_df.iloc[:-time_shift]

In [72]:
data

,time,ft_11402000,ft_11318500,ft_11266500,ft_11208000,ft_11202710,ft_11185500,ft_11189500,target
10958,2014-10-02,16.6,2.00,11.0,1.5,13.0,0.0,2.27,16.6
10959,2014-10-03,18.0,2.00,10.8,1.4,13.0,0.0,2.21,18.0
10960,2014-10-04,17.3,2.10,10.7,1.5,13.0,0.0,2.20,17.3
10961,2014-10-05,16.0,2.30,10.3,1.5,13.0,0.0,2.22,16.0
10962,2014-10-06,17.4,2.60,10.1,1.5,12.0,0.0,2.16,17.4
...,...,...,...,...,...,...,...,...,...
12383,2018-08-27,17.0,5.82,40.2,4.1,22.0,55.0,2.79,17.0
12384,2018-08-28,19.2,5.72,38.6,4.1,23.0,55.0,2.75,19.2
12385,2018-08-29,18.6,5.87,37.7,4.0,23.0,55.0,2.77,18.6
12386,2018-08-30,20.0,5.94,36.2,4.1,23.0,52.0,2.48,20.0


In [76]:
feature_cols = data.drop(columns = ['time','target']).columns.tolist()
feature_cols

['ft_11402000',
 'ft_11318500',
 'ft_11266500',
 'ft_11208000',
 'ft_11202710',
 'ft_11185500',
 'ft_11189500']

In [73]:
test_head = data.index[int(0.8*len(data))]

df_train = gage_df.loc[:test_head,:]
df_test = gage_df.loc[test_head:,:]
target_train = target_data.loc[:test_head]
target_test = target_data.loc[test_head:]

In [124]:
import torch
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[target].values).float()
        self.X = torch.tensor(dataframe[features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

ModuleNotFoundError: No module named 'torch'

In [77]:
i = 27
sequence_length = 7
features = feature_cols
target = 'target'

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)

X, y = train_dataset[i]
print(X)

tensor([[22.5000,  2.6000, 10.6000,  1.4000, 14.0000,  0.0000,  2.6600],
        [22.5000,  2.7000, 10.5000,  1.4000, 13.0000,  0.0000,  3.1500],
        [23.2000,  3.3000, 10.4000,  1.4000, 13.0000,  0.0000,  3.0700],
        [35.5000,  3.7000, 10.5000,  1.5000, 14.0000,  0.0000,  3.0300],
        [35.0000,  3.7000, 10.7000,  1.5000, 14.0000,  0.0000,  3.1500],
        [28.1000,  3.4000, 10.6000,  1.5000, 14.0000,  0.0000,  3.2200],
        [26.3000,  3.2000, 10.7000,  1.5000, 14.0000,  0.0000,  3.2400]])


In [78]:
from torch.utils.data import DataLoader
torch.manual_seed(42)

train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)

X, y = next(iter(train_loader))
print(X.shape)
print(X)

torch.Size([3, 7, 7])
tensor([[[6.9600e+02, 1.7600e+02, 6.0800e+03, 3.4000e+00, 4.0800e+02,
          5.6500e+02, 1.2500e+03],
         [6.6400e+02, 1.7000e+02, 5.6900e+03, 3.0000e+00, 4.3800e+02,
          5.7600e+02, 1.2400e+03],
         [5.9100e+02, 1.6400e+02, 3.9200e+03, 3.0000e+00, 4.1100e+02,
          5.6000e+02, 1.2300e+03],
         [5.3200e+02, 1.5800e+02, 3.4400e+03, 1.3000e+00, 3.7500e+02,
          5.5300e+02, 1.1600e+03],
         [4.8600e+02, 1.5500e+02, 3.0700e+03, 0.0000e+00, 3.4000e+02,
          5.5000e+02, 1.0800e+03],
         [4.6300e+02, 1.5400e+02, 2.5800e+03, 0.0000e+00, 3.0100e+02,
          5.3000e+02, 1.0200e+03],
         [4.7200e+02, 1.5300e+02, 2.3800e+03, 3.1000e+00, 2.8300e+02,
          5.2700e+02, 9.7500e+02]],

        [[3.6700e+01, 3.8000e+00, 4.0100e+02, 1.1000e+01, 6.1000e+00,
          1.9900e+02, 8.7700e+00],
         [3.8800e+01, 7.8000e+00, 3.7800e+02, 1.1000e+01, 6.3000e+00,
          1.8900e+02, 8.6500e+00],
         [4.4800e+01, 1.6000e+0

In [79]:
batch_size = 4
sequence_length = 7

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    df_test,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([4, 7, 7])
Target shape: torch.Size([4])


In [80]:
from torch import nn

class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 1

        self.lstm = nn.LSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()

        _, (hn, _) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out

In [81]:
learning_rate = 5e-4
num_hidden_units = 8

model = ShallowRegressionLSTM(num_sensors=len(features), hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [82]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()

    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")

def test_model(data_loader, model, loss_function):

    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")


print("Untrained test\n--------")
test_model(test_loader, model, loss_function)
print()

for ix_epoch in range(30):
    print(f"Epoch {ix_epoch}\n---------")
    train_model(train_loader, model, loss_function, optimizer=optimizer)
    test_model(test_loader, model, loss_function)
    print()

Untrained test
--------
Test loss: 170873.41584797148

Epoch 0
---------
Train loss: 677741.1989021966
Test loss: 170767.7034095088

Epoch 1
---------
Train loss: 677501.5249025564
Test loss: 170605.03493953656

Epoch 2
---------
Train loss: 677299.4300107524
Test loss: 170366.73983841907

Epoch 3
---------
Train loss: 676706.1167311452
Test loss: 170094.44985778423

Epoch 4
---------
Train loss: 676884.1327460951
Test loss: 169811.28410793256

Epoch 5
---------
Train loss: 675669.3594616082
Test loss: 169516.9464481209

Epoch 6
---------
Train loss: 675072.644075799
Test loss: 169140.05310087567

Epoch 7
---------
Train loss: 674459.0158398525
Test loss: 168847.5033165171

Epoch 8
---------
Train loss: 673941.1603868598
Test loss: 168567.038689722

Epoch 9
---------
Train loss: 673500.5841700324
Test loss: 168269.523753637

Epoch 10
---------
Train loss: 673055.2345612955
Test loss: 168014.8466104435

Epoch 11
---------
Train loss: 672818.6869991715
Test loss: 167716.416280167

Epoch 

In [83]:
def predict(data_loader, model):

    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)

    return output


train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

ystar_col = "Model forecast"
df_train[ystar_col] = predict(train_eval_loader, model).numpy()
df_test[ystar_col] = predict(test_loader, model).numpy()

df_out = pd.concat((df_train, df_test))[[target, ystar_col]]

# for c in df_out.columns:
#     df_out[c] = df_out[c] * target_stdev + target_mean

print(df_out)

       target  Model forecast
10958    16.6       18.184389
10959    18.0       18.563141
10960    17.3       18.346769
10961    16.0       18.009441
10962    17.4       18.667486
...       ...             ...
12413    23.1       17.656294
12414    22.8       17.612049
12415    20.9       17.542467
12416    17.4       17.420143
12417    18.4       17.962250

[1461 rows x 2 columns]


<ipython-input-83-eac2ac85467c>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[ystar_col] = predict(train_eval_loader, model).numpy()
<ipython-input-83-eac2ac85467c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[ystar_col] = predict(test_loader, model).numpy()


In [84]:
df_out

,target,Model forecast
10958,16.6,18.184389
10959,18.0,18.563141
10960,17.3,18.346769
10961,16.0,18.009441
10962,17.4,18.667486
...,...,...
12413,23.1,17.656294
12414,22.8,17.612049
12415,20.9,17.542467
12416,17.4,17.420143
